# Day4 - synchrotron 2015 training advanced - group 2

Topics today:

 - logging
 - shell programming
 - low level debugging using pdb
 - databases
 - Web programming
     - requests library
     - Theory of web
     - REST concept
     - Flask
 - goodbye and good luck!

# Logging

http://www.shutupandship.com/2012/02/how-python-logging-module-works.html

  - Handlers are the actual places we output to.
  - Loggers are the named 'bucket' objects we create and write to in our code.
  - There can be many handlers attached to one logger.
  - Messages sent to Loggers propagate to parent loggers (based on . syntax with '' root logger at the top) unless you supress.
  - Error level on both the logger and handler will prevent messages of a lower value getting through (lowest is debug, then info, error is a high value)

Official doco https://docs.python.org/2/library/logging.html

## Simplest way of using logging

You don't have to create an instance of anything.  Just use the "root logger"

In [1]:
import logging

In [3]:
logging.info("this is a message via the logging system")

We can't see anything yet because the default logging levels is 'warning' thus we don't see messages lower than warning.

These are the logging levels.

    CRITICAL
    ERROR
    WARNING <<<------
    INFO
    DEBUG
    NOTSET

In [6]:
logging.warning("this should get through")
logging.error("this shoudl of course")
logging.debug("will not")

ERROR:root:this shoudl of course


In [14]:
logging.basicConfig(level=logging.DEBUG)  # TODO an exercise to get this to work
logging.debug("we will see this now, if we are lucky")

## proper advanced use of logging 

In [15]:
# using you own logger instead of the root logger

import logging
log = logging.getLogger('mayapp2')
log.setLevel(logging.INFO)

# use the logger
log.info('this is a piece of information')

INFO:mayapp2:this is a piece of information


Typically you use the ```__name__``` as the name of your logger thus you get the file name which geenrated the log message in your log messages. 

In [20]:
log2 = logging.getLogger(__name__)
log2.setLevel(logging.INFO)
log2.info('hello')

INFO:__main__:hello


In [16]:
# changing the log level on the fly
log.setLevel(logging.ERROR)
log.info("this won't be printed")

In [17]:
# Andy's useful function to show what levels in a logger are set

def report_level(o):
    # pass in a logger
    print 'reporting level for', o
    print 'DEBUG', o.isEnabledFor(logging.DEBUG)
    print 'INFO', o.isEnabledFor(logging.INFO)
    print 'WARN', o.isEnabledFor(logging.WARN)
    print 'ERROR', o.isEnabledFor(logging.ERROR)
    print 'CRITICAL', o.isEnabledFor(logging.CRITICAL)
report_level(log)

reporting level for <logging.Logger object at 0x104846210>
DEBUG False
INFO False
WARN False
ERROR True
CRITICAL True


## wiring in handlers to loggers

<img src="logging_whiteboard1.jpeg">
<img src=https://raw.githubusercontent.com/abulka/python_train2015_adv_v2/master/logging_whiteboard1.jpeg>

In [22]:
# handler that writes to a file
fh = logging.FileHandler('debug.log')

# add the handler to the logger
log.addHandler(fh)

# test
log.error('this is an error mate - today is 15th')

ERROR:mayapp2:this is an error mate - today is 15th


List of handlers built into python https://docs.python.org/2/library/logging.handlers.html#module-logging.handlers

**tip**

    tail -f debug.log
    
will print the end of the log file in real time - handy for watching log files

## adding formatting

In [28]:
# create a formatter and set the formatter for the handler, 
# then attach to our logger
frmt = logging.Formatter('LOG AHOY %(asctime)s - %(name)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s')
fh = logging.FileHandler('debug.log')  
fh.setFormatter(frmt)  # WIRING

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
log.addHandler(fh)  # WIRING

# Test
log.error('this is a nicer looking message')
log.error('one more for the road')


ERROR:__main__:this is a nicer looking message
ERROR:__main__:one more for the road


**Exercise**: Create a logger which outputs to **two** handlers simultaneously.  They are both file handlers 'simple.log' and 'debug.log'.  Make the formatter for them different, so the simple one gets a simpler format.  Also, the simple handler should only get messages at info level and above, whilst the other hander gets all debug messages.

In [30]:
# Solution

# two formatters
fmt_complex = logging.Formatter('LOG AHOY %(asctime)s - %(name)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s')
fmt_simple = logging.Formatter('%(message)s')

# two handlers
handler_simple = logging.FileHandler('simple.log')  
handler_complex = logging.FileHandler('debug.log')
# wiring formatter to the handler
handler_simple.setFormatter(fmt_simple)
handler_complex.setFormatter(fmt_complex)

# one logger!
log = logging.getLogger('main logger') # call a logger anything you like
log.addHandler(handler_simple)  # WIRING
log.addHandler(handler_complex)  # WIRING

# set levels
log.setLevel(logging.DEBUG)   # Allow everything through
handler_simple.setLevel(logging.INFO)  # only allow through >= INFO
handler_complex.setLevel(logging.DEBUG) # allow through >= DEBUG

# Test
log.error('this is a nice looking message')
log.error('and one more for the road')


ERROR:main logger:this is a nice looking message
ERROR:main logger:and one more for the road


### Discussion
- other modules can talk to the same logger (the get Logger will find an existing logger or create a new one)
- OR other modules can create their own logger and as long as you are wiring that logger to the same handlers, the output will appear in the same place, except you will have the different module name being reported

** inheritance tip**
- use inhertance to save yourself from having to wire up each new logger. e.g. a logger called 'a' can be wired up.  Then create a logger 'a.b' and it will inherit all the wiring.


Many web frameworks etc. have a way to set up your logging via dictionaries (configuration) rather than explicit wiring.  E.g. DJANGO 

Third party logging services - https://logentries.com/product/live-tail/

# Shell programming

In [33]:
import os
results = os.popen("ls").readlines()
for line in results:
    if 'jpeg' in line:
        print 'GOT:', line.rstrip()

GOT: logging_whiteboard1.jpeg


In [34]:
# if you don't want the actual output of the commands, just the 'success state'
# 0 is success (by convention)
# 1 is error
import os
success = os.system("ls")  # this only gets you the state, not the outut
print success

0


## you can write programs to return status codes:

Write your 'external' program in bash, or python - or any language

**sayhello.sh**
    # sample bash command
    echo "hello"
    exit 0
    
**sayhello.py**
    #!/usr/bin/python
    import sys
    print "hello"
    sys.exit(1)                                                           

The ```echo $?``` tells you the status code of the last run program

    bash-3.2$ python sayhello.py
    hello
    bash-3.2$ echo $?                                                         
    0                                                                     
    bash-3.2$ ./sayhello.sh
    hello
    bash-3.2$ echo $?
    0
    
Then we alter the code so that it has a different exit statusexit
    bash-3.2$ ./sayhello.sh
    hello
    bash-3.2$ echo $?
    1                                                                     
    

In [47]:
# Example of calling an external program and checking the status code.
# In this case we raise a custom exception if there is an error.

class MyOsCallingException(Exception):
    pass

exit_status = os.system("./sayhello.sh")
#exit_status = os.system("./sayhello.py")
if exit_status == 0:
    print 'command succeeded'
else:
    # raise RuntimeError("my external program run failed")
    raise MyOsCallingException("my external program run failed")
    

command succeeded


In [41]:
# need the ./ so that the command is found 
# (unless you have . in your PATH or this command lived in a dir 
# that is in your paths)
print os.popen("./sayhello.sh").readlines()

['hello\n']


In [42]:
# PATH issues are similar to PYTHONPATH issues
import sys
import pprint
pprint.pprint(sys.path)

['',
 '/Users/Andy/miniconda/envs/py2k/lib/python27.zip',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/plat-darwin',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/plat-mac',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/lib-tk',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/lib-old',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/lib-dynload',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/site-packages',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/site-packages/setuptools-5.7-py2.7.egg',
 '/Users/Andy/miniconda/envs/py2k/lib/python2.7/site-packages/IPython/extensions']


## subprocess module

In [48]:
# get output of a command using the subprocess module technique
import subprocess

# Run sheell command
result = subprocess.check_output(["ps", "-e", "-a", "-f"])
print result[0:400]  # limit what we display

  UID   PID  PPID   C STIME   TTY           TIME CMD
    0     1     0   0 Sun09am ??         1:28.96 /sbin/launchd
    0    44     1   0 Sun09am ??         0:07.40 /usr/sbin/syslogd
    0    45     1   0 Sun09am ??         0:30.79 /usr/libexec/UserEventAgent (System)
    0    47     1   0 Sun09am ??         0:05.62 /usr/libexec/kextd
    0    48     1   0 Sun09am ??         0:25.50 /System/Librar


In [44]:
p = subprocess.Popen("ls -lR", shell=True)
print p

In [46]:
process = subprocess.Popen(['ls', '-l'], stdout=subprocess.PIPE)
print process
#process.wait()
s = process.stdout.read()

More info:

http://www.python-course.eu/os_module_shell.php

# Databases

other 'persistence' options:

- **redis** (in memory, saved itslef automatically, non sql - 'advanced key value store', where values can be binary files, network endpoint, supports lists and sets as values, fast, publisher subscribe so can see changed when push to it, json support
- **memcached** - in memory dictionary which is globally accessible on the one machine / has server (like redis but cheap and doesn't persist)
- **pickle** (saves python objects to disk and back again)
- **shelve** (dictionary that persists to disk automatically - like redis but sinple python object and non network)
- use **files** - manually write and read, possibly use csv format, 
- **DB (sql based)** - standard, usable by other systems, relational, queriable etc.

## sqlite

In [51]:
import sqlite3
print sqlite3.sqlite_version

3.8.4.1


In [52]:
# classic python

class Car:
    def __init__(self, id, name, price):
        self.id = id
        self.name = name
        self.price = price
        
audi = Car(1, 'Audi', 120000)
print audi

<__main__.Car instance at 0x104853dd0>


In [54]:
# SQL DB equivalent

con = sqlite3.connect('cars.db')
with con:
    cur = con.cursor()
    cur.execute("create table Cars(id INT, name TEXT, price INT)") # create table
    cur.execute("insert into Cars values(1, 'Audi', 120000)")
print 'all done'

all done


Access your db from the command line:

    > sqlite3 cars.db
    select * from Cars
    .tables
    .schema Cars

etc.


sqllite is part of python distibution, no need to install anything.

https://github.com/lazierthanthou/sqlite-manager  visual tool for firefox, which lets you look inside databases



In [60]:
con = sqlite3.connect('cars.db')
with con:
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute("select * from Cars")
    rows = cur.fetchall()
    
    for row in rows:
        print "%s %s %s" % (row['id'], row['name'], row['price'])    

1 Audi 120001
2 Commodore 40000
10 Tesla 130000
10 Tesla 130000
10 Tesla 130000


In [59]:
# leave out the create table command and insert more values!
with con:
    cur = con.cursor()
    cur.execute("insert into Cars values(10, 'Tesla', 130000)")

## ORM - object relational mapping

Rave on Anders and having sql like powers built into the language - rather than passing strings to a black box sql engine.
https://msdn.microsoft.com/en-us/library/bb397947.aspx

e.g.

    IEnumerable<CustomerTuple> locals = 
        customers.Where(c => c.ZipCode == 91822)
                     .Select(c => new CustomerTuple(c.Name, c.Address));
                 
                 

### HOME GROWN ORM

In [ ]:
# HOME GROWN ORM  :-)

# load all the db cars into a list of Car objects

with con:
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute("select * from Cars")
    rows = cur.fetchall()
    
    cars = []
    for row in rows:
        car = Car(row['id'], row['name'], row['price'])
        cars.append(car)
        
print "We have loaded", len(cars), "cars from the db"
for car in cars:
    print "%s %s %s" % (car.id, car.name, car.price)
    
# doing search using pure python
print 'cheaper cars:'
for car in cars:
    if car.price < 60000:
        print car.name


# PDB debugging 

In [ ]:
import pdb; pdb.set_trace()

**KEY COMMANDS TO USE**

 - use 'c' to continue
 - use 'l' to list source code around current position
 - 'n' next (step over - which doesn't skip anything, just doesn't bother drilling in to a function)
 - 's' step (step in - drill into a function step by step)
 - 'q' quit without running any more python code

Note you can use any python command - you are inside the python interpreter!! :-)

Help on any command e.g.

    h b
help on the break command.

# WEB programming

http is text based - its a 'language' of sorts

GET, POST, PUT, DELETE talk to urls

request and response

    response codes e.g.
    200, ok
    202 here is your response but I'm not finished so check back later,
    404 not found, 
    500 server error, 
    301 redirect etc. 

see http://en.wikipedia.org/wiki/List_of_HTTP_status_codes#2xx_Success for all codes.  These appear in the HEADER of the repsonse.

header vs body

## Using httpie or curl to invoke URLs

use curl
curl -i to see the headers

or

httpie (brew install httpie or use sudo ...., pip install httpie) https://github.com/jakubroztocil/httpie

invoke with 

    http ......

http www.google.com

    HTTP/1.1 302 Found
    Alternate-Protocol: 80:quic,p=0.5
    Cache-Control: private
    Content-Length: 262
    Content-Type: text/html; charset=UTF-8
    Date: Mon, 30 Mar 2015 01:28:17 GMT
    Location: http://www.google.com.au/?gfe_rd=cr&ei=saYYVc2KLrDu8we9_oD4CA
    Server: GFE/2.0

    <HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
    <TITLE>302 Moved</TITLE></HEAD><BODY>
    <H1>302 Moved</H1>
    The document has moved
    <A HREF="http://www.google.com.au/?gfe_rd=cr&amp;ei=saYYVc2KLrDu8we9_oD4CA">here</A>.
    </BODY></HTML>


## Simple FLASK server

In [ ]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello world'

@app.route('/hello'):
def hello_world2():
    return 'hello world 2 you attempt to do a %s on my url' % request.method

if __name__ == '__main__':
    app.run(debug=True)  # the server will automatically reload when source code changes
    

http localhost:5000/hello name=fred

    HTTP/1.0 405 METHOD NOT ALLOWED
    Allow: HEAD, OPTIONS, GET
    Content-Length: 178
    Content-Type: text/html
    Date: Mon, 15 Jun 2015 05:15:07 GMT
    Server: Werkzeug/0.10.1 Python/2.7.8

    <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
    <title>405 Method Not Allowed</title>
    <h1>Method Not Allowed</h1>
    <p>The method is not allowed for the requested URL.</p>


## REST - what does it mean?

Basically treating URLs as 'resources' that are accessed via CRUD operations - which http happens to have!  Server should be stateless and discoverable etc.

<img src="REST_whiteboard2.jpeg">
<img src=https://raw.githubusercontent.com/abulka/python_train2015_adv_v2/master/REST_whiteboard2.jpeg>

## Getting info UP to the server

### technique 1 - use a GET and use your url structure


In [ ]:
# http://127.0.0.1:5000/hello2/fred

@app.route('/hello2/<name>', methods=['GET'])
def hello_world2(name=None):
    return "hello %s" % name

# Ps can route similar url which has differently typed url components (e.g. int vs string) to different functions e.g.
@app.route('/hello2/<int:name>')
def hello_world2_int(name=None):
    res = ""
    num_times = name  # since a number was passed in
    for i in range(num_times):
        res += "hello! "
    return res

# The point is, the above urls are the same
# you might get a string one day, or an int 
# and flask will route you to different methods depending on the type.

### technique 2 - use a GET and use get parameters

http://127.0.0.1:5000/hello3?who=fred
http://127.0.0.1:5000/hello3?who=fred&when=tomorrow  # two name value pairs

NOTE: be careful to not veer off into non restful land and use GET for sending real info.  Its ok if you use it for e.g. ?limit=100&filter="age > 100"

To access parameters submitted in the URL (?key=value) you can use the args attribute:

thus

http://127.0.0.1:5000/hello3?who=mary

In [ ]:
# Build a handler for GET where you can send up parameters using the ? 
# syntax above

#The method you will need is:
request.args.get

# and remember to
from flask import request

In [ ]:
# technique 2
# To access parameters submitted in the URL (?key=value) you can use the args attribute:
@app.route('/hello3', methods=['GET'])
def hello_world3():
    who = request.args.get('who', 'stranger')  # if parameter not found
    return "hello %s" % who


testing...

    (py2k)Andys-MacBook-Air:python_training_2015_advanced Andy$ http localhost:5000/hello3?who=fred
    HTTP/1.0 200 OK
    Content-Length: 10
    Content-Type: text/html; charset=utf-8
    Date: Mon, 15 Jun 2015 05:57:07 GMT
    Server: Werkzeug/0.10.1 Python/2.7.8

    hello fred

    (py2k)Andys-MacBook-Air:python_training_2015_advanced Andy$ http localhost:5000/hello3
    HTTP/1.0 200 OK
    Content-Length: 14
    Content-Type: text/html; charset=utf-8
    Date: Mon, 15 Jun 2015 05:57:23 GMT
    Server: Werkzeug/0.10.1 Python/2.7.8

    hello stranger


### technique 3 - get info passed via a POST

In [ ]:
# using requests library to send up a form

>>> import requests
>>> response = requests.post('http://localhost:5000/info1', data={'name':'fred'})


Using curl to send up a form

    curl -F name=fred 127.0.0.1:5000/info1


In [ ]:
# technique 3 - get the info via a POST
@app.route('/info1', methods=['POST'])
def info1():
    if request.method == 'POST':
        print request.form
    return 'Name is %s' % request.form['name']


In [ ]:

@app.route('/info1', methods=['POST'])
def info1():
    # request.data is a string
    if request.method == 'POST':
        print 'got a post!!!', request.content_type
        # import pdb;pdb.set_trace()

        # universal - but only returns a string.
        #request.get_data()

        # if content type sent from a form
        # e.g. http http://127.0.0.1:5000/info1 a=b -f
        if '/json' in request.content_type:
            # if content type sent as json
            # http http://127.0.0.1:5000/info1 a=b
            # print request.get_json()['a']
            return "got id=%(a)s" % request.get_json()
        else:
            # sent as a form

            # be careful in case all the form data is not supplied and keys are missing
            # if 'a' in request.form.keys():
            #     ....

            # print request.form['a']
            return "got id=%(a)s" % request.form



# Flask - templates

In [ ]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/hello')
def hello_world():
    return 'Hello world'

@app.route('/hello/<name>', methods=['GET', 'POST'])
def hello_world2(name=None):
    return 'hello world %s you attempt to do a %s on my url' % (name, request.method)

# Ps can route similar url which has differently typed url components (e.g. int vs string) to different functions e.g.
@app.route('/hello/<int:name>')
def hello_world2_int(name=None):
    res = ""
    num_times = name  # since a number was passed in
    for i in range(num_times):
        res += "hello! "
    return res

# technique 2
# To access parameters submitted in the URL (?key=value) you can use the args attribute:
@app.route('/hello3', methods=['GET'])
def hello_world3():
    who = request.args.get('who', 'stranger')  # if parameter not found
    return "hello %s" % who


# technique 3 - get the info via a POST
@app.route('/info1', methods=['POST'])
def info1():
    if request.method == 'POST':
        print request.form
    return 'Name is %s' % request.form['name']


from flask import render_template
@app.route('/template1')
def template1():
    return render_template('template1.html',
                           info='here is some info',
                           num_times=20,
                           )

from flask import render_template
@app.route('/cars')
def cars():

    class Car:
        def __init__(self, id, name, price):
            self.Id = id
            self.Name = name
            self.Price = price

    cars = [
        Car(1, 'VW', 5000),
        Car(2, 'Mercedes', 210000),
        Car(3, 'Jag', 110000) ]

    return render_template('template2.html',
                           cars=cars)



if __name__ == '__main__':
    app.run(debug=True)  # the server will automatically reload when source code changes


### corresponding templates:

templates/template1.html

In [ ]:
<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title></title>
</head>
<body>
  Hello world AGAIN!
 <p>this is some <b>{{ info }}</b></p>

{% for i in range(num_times)  %}
    <p>{{ i }}</p>
{% endfor %}


</body>
</html>

In [ ]:
templates/template2.html

Creating Car objects on the server and then display them via a template:

In [ ]:
<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title></title>
</head>
<body>

<h1>Here are my Cars</h1>
{% for car in cars  %}
    <p>Id: {{ car.Id }}</p>
    <p>Name: {{ car.Name }}</p>
    <p>Price: {{ car.Price }}</p>
{% endfor %}

</body>
</html>

http://127.0.0.1:5000/cars

**result:**

Here are my Cars

Id: 1

Name: VW

Price: 5000

Id: 2

Name: Mercedes

Price: 210000

Id: 3

Name: Jag

Price: 110000

See uploaded project **flask_demo2**

END OF 4 day course.  Thank you.